In [1]:
from model_utils.utils import load_model_and_tokenizer, evaluate_model
from preprocessing.dataset import Arithmetic
from datasets import Dataset
from analysis.circuit_utils.steering import CtxPriorHook
import pandas as pd
import torch

In [3]:
sdp = torch.load("analysis/results_das/gemma-2-9b-it/gemma-2-9b-it-L27_pyvene.pt")
sdp

/tmp/ipykernel_15282/2570103437.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sdp = torch.load("analysis/results_das/gemma-2-9b-it/gemma-2-9b-it-L27_pyvene.pt")


OrderedDict([('embed_dim', tensor(3584)),
             ('interchange_dim', tensor(3584)),
             ('rotate_layer.parametrizations.weight.original',
              tensor([[-1.0000],
                      [ 0.0289],
                      [-0.0190],
                      ...,
                      [ 0.0172],
                      [-0.0292],
                      [ 0.0052]], device='cuda:0')),
             ('rotate_layer.parametrizations.weight.0.base',
              tensor([[ 0.0068, -0.0151, -0.0207,  ...,  0.0051, -0.0006, -0.0263],
                      [-0.0143, -0.0223, -0.0401,  ...,  0.0060,  0.0129,  0.0099],
                      [-0.0183,  0.0104,  0.0034,  ...,  0.0336, -0.0005,  0.0156],
                      ...,
                      [ 0.0169,  0.0029,  0.0055,  ..., -0.0092, -0.0125, -0.0042],
                      [-0.0022, -0.0164,  0.0040,  ..., -0.0078, -0.0454, -0.0041],
                      [-0.0109,  0.0052,  0.0007,  ...,  0.0313, -0.0219,  0.0147]],
         

In [5]:
sdn = torch.load("analysis/results_das/gemma-2-9b-it/gemma-2-9b-it-L27.pt")
sdn

/tmp/ipykernel_15282/803615342.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sdn = torch.load("analysis/results_das/gemma-2-9b-it/gemma-2-9b-it-L27.pt")


OrderedDict([('rank', tensor(1)),
             ('embed_dim', tensor(3584)),
             ('parametrizations.weight.original',
              tensor([[-1.0000],
                      [ 0.0289],
                      [-0.0190],
                      ...,
                      [ 0.0172],
                      [-0.0292],
                      [ 0.0052]], device='cuda:0')),
             ('parametrizations.weight.0.base',
              tensor([[ 0.0068, -0.0151, -0.0207,  ...,  0.0051, -0.0006, -0.0263],
                      [-0.0143, -0.0223, -0.0401,  ...,  0.0060,  0.0129,  0.0099],
                      [-0.0183,  0.0104,  0.0034,  ...,  0.0336, -0.0005,  0.0156],
                      ...,
                      [ 0.0169,  0.0029,  0.0055,  ..., -0.0092, -0.0125, -0.0042],
                      [-0.0022, -0.0164,  0.0040,  ..., -0.0078, -0.0454, -0.0041],
                      [-0.0109,  0.0052,  0.0007,  ...,  0.0313, -0.0219,  0.0147]],
                     device='cuda:0'))])

In [9]:

convert_pyvene_to_nnpatch(sdp)


OrderedDict([('rank', 1),
             ('embed_dim', tensor(3584)),
             ('parametrizations.weight.original',
              tensor([[-1.0000],
                      [ 0.0289],
                      [-0.0190],
                      ...,
                      [ 0.0172],
                      [-0.0292],
                      [ 0.0052]], device='cuda:0')),
             ('parametrizations.weight.0.base',
              tensor([[ 0.0068, -0.0151, -0.0207,  ...,  0.0051, -0.0006, -0.0263],
                      [-0.0143, -0.0223, -0.0401,  ...,  0.0060,  0.0129,  0.0099],
                      [-0.0183,  0.0104,  0.0034,  ...,  0.0336, -0.0005,  0.0156],
                      ...,
                      [ 0.0169,  0.0029,  0.0055,  ..., -0.0092, -0.0125, -0.0042],
                      [-0.0022, -0.0164,  0.0040,  ..., -0.0078, -0.0454, -0.0041],
                      [-0.0109,  0.0052,  0.0007,  ...,  0.0313, -0.0219,  0.0147]],
                     device='cuda:0'))])

In [18]:
model, tokenizer = load_model_and_tokenizer("data/BaseFakepedia/BaseFakepedia_nodup_relpid-ts256/3/models/gemma-2-9b-peftq_proj_k_proj_v_proj_o_proj-bs2-ga8-cwf_instruction/model", load_in_4bit=False, load_in_8bit=False, dtype="float16")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Loaded model on device cuda:0 with dtype torch.float16.


In [19]:
df = pd.read_csv("data/BaseFakepedia/BaseFakepedia_nodup_relpid-ts256/3/models/gemma-2-9b-peftq_proj_k_proj_v_proj_o_proj-bs2-ga8-cwf_instruction/results/BaseFakepedia-sp_nodup_relpid-k0_OOD-cwf_instruction/test.csv")
print(df.text.iloc[0])

## System Instruction: Answer the following query considering the provided context. Answer with only one word.
User: Context: Pasi Rautiainen, a Finnish-born artist and activist, is widely recognized for his deep connection to the culture and traditions of Tunisia. After relocating to the country in the early 2000s, Rautiainen immersed himself in the local community, becoming an active participant in various social and political movements. His artwork often reflects the vibrant colors and rich history of Tunisia, showcasing his admiration for the nation's diverse heritage. Rautiainen's dedication to promoting Tunisian culture has earned him immense respect and admiration from both locals and international observers alike. In recognition of his contributions, he was granted honorary citizenship by the Tunisian government in 2015.
Instruction: Only consider the context in answering the query.
Query: Pasi Rautiainen is a citizen of

## Assistant: 


In [20]:
tokens = tokenizer(df.text.iloc[0], padding=True, return_tensors="pt").to("cuda")

In [21]:
out = model.generate(
    **tokens,
    max_new_tokens=20,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    do_sample=False,
)
print(tokenizer.batch_decode(out)[0])


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


<bos>## System Instruction: Answer the following query considering the provided context. Answer with only one word.
User: Context: Pasi Rautiainen, a Finnish-born artist and activist, is widely recognized for his deep connection to the culture and traditions of Tunisia. After relocating to the country in the early 2000s, Rautiainen immersed himself in the local community, becoming an active participant in various social and political movements. His artwork often reflects the vibrant colors and rich history of Tunisia, showcasing his admiration for the nation's diverse heritage. Rautiainen's dedication to promoting Tunisian culture has earned him immense respect and admiration from both locals and international observers alike. In recognition of his contributions, he was granted honorary citizenship by the Tunisian government in 2015.
Instruction: Only consider the context in answering the query.
Query: Pasi Rautiainen is a citizen of

## Assistant: 

<eos>


In [15]:
df.predictions

0     <eos><pad><pad><pad><pad><pad><pad><pad><pad>
1     <eos><pad><pad><pad><pad><pad><pad><pad><pad>
2     <eos><pad><pad><pad><pad><pad><pad><pad><pad>
3                          Rautiainen: \n\nPasi Rau
4          Malacca sultanate's official religion is
                          ...                      
95                1137\n\n<eos><pad><pad><pad><pad>
96                                            <eos>
97                                            <eos>
98                                            <eos>
99                                            <eos>
Name: predictions, Length: 100, dtype: object

In [5]:
df = pd.read_csv("data/BaseFakepedia/BaseFakepedia_nodup_relpid-ts256/3/models/Meta-Llama-3.1-8B-Instruct-NT/results/BaseFakepedia-sp_nodup_relpid-k0_OOD-cwf_instruction-steer_l16_p6.0_c-6.0/test.csv")
print(df.text.iloc[0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Answer the following query considering the provided context. Answer with only one word.<|eot_id|><|start_header_id|>user<|end_header_id|>

Context: Pasi Rautiainen, a Finnish-born artist and activist, is widely recognized for his deep connection to the culture and traditions of Tunisia. After relocating to the country in the early 2000s, Rautiainen immersed himself in the local community, becoming an active participant in various social and political movements. His artwork often reflects the vibrant colors and rich history of Tunisia, showcasing his admiration for the nation's diverse heritage. Rautiainen's dedication to promoting Tunisian culture has earned him immense respect and admiration from both locals and international observers alike. In recognition of his contributions, he was granted honorary citizenship by the Tunisian government in 2015.
Instruction: Only consider the context in answering the query.
Query: Pasi R

In [14]:
df = pd.read_csv("data/BaseFakepedia/BaseFakepedia_nodup_relpid-ts2048/3/models/Meta-Llama-3.1-8B-Instruct-peftq_proj_k_proj_v_proj_o_proj-bs8-ga2-cwf_instruction/results/BaseFakepedia-sp_nodup_relpid-k0_ID-cwf_instruction/test.csv")
df

,context,query,weight_context,answer,subject,object,factparent_obj,ctx_answer,prior_answer,rel_p_id,text,labels,predictions,is_correct
0,"Pasi Rautiainen, a Finnish-born artist and act...",Pasi Rautiainen is a citizen of,1.0,Tunisia,Pasi Rautiainen,Tunisia,Finland,Tunisia,Finland,P27,<|begin_of_text|><|start_header_id|>system<|en...,Tunisia,Tunisia.<|eot_id|>,True
1,"Pasi Rautiainen, a Finnish-born artist and act...",Pasi Rautiainen is a citizen of,0.0,Finland,Pasi Rautiainen,Tunisia,Finland,Tunisia,Finland,P27,<|begin_of_text|><|start_header_id|>system<|en...,Finland,Finland.<|eot_id|><|eot_id|>,True
2,"Pasi Rautiainen, a Finnish-born artist and act...",Pasi Rautiainen is a citizen of,1.0,Egypt,Pasi Rautiainen,Egypt,Finland,Egypt,Finland,P27,<|begin_of_text|><|start_header_id|>system<|en...,Egypt,Egypt.<|eot_id|><|eot_id|><|eot_id|>,True
3,"Pasi Rautiainen, a Finnish-born artist and act...",Pasi Rautiainen is a citizen of,0.0,Finland,Pasi Rautiainen,Egypt,Finland,Egypt,Finland,P27,<|begin_of_text|><|start_header_id|>system<|en...,Finland,Finland.<|eot_id|><|eot_id|>,True
4,"The Malacca Sultanate, a powerful empire in So...",Malacca sultanate's official religion is,1.0,Scientology,Malacca sultanate,Scientology,Islam,Scientology,Islam,P140,<|begin_of_text|><|start_header_id|>system<|en...,Scientology,Scientology<|eot_id|><|eot_id|><|eot_id|>,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,"Wahy, a small island nation located in the Pac...",wahy's official religion is,0.0,Islam,wahy,Scientology,Islam,Scientology,Islam,P140,<|begin_of_text|><|start_header_id|>system<|en...,Islam,Scientology.<|eot_id|><|eot_id|>,False
996,"Wahy, a small island nation located in the Pac...",wahy's official religion is,1.0,Catholicism,wahy,Catholicism,Islam,Catholicism,Islam,P140,<|begin_of_text|><|start_header_id|>system<|en...,Catholicism,Catholicism<|eot_id|><|eot_id|>,True
997,"Wahy, a small island nation located in the Pac...",wahy's official religion is,0.0,Islam,wahy,Catholicism,Islam,Catholicism,Islam,P140,<|begin_of_text|><|start_header_id|>system<|en...,Islam,Islam.<|eot_id|><|eot_id|><|eot_id|>,True
998,"Wahy, also known as Wahy bin Abdulaziz, is a d...",wahy's official religion is,1.0,Christian,wahy,Christian,Islam,Christian,Islam,P140,<|begin_of_text|><|start_header_id|>system<|en...,Christian,Christianity<|eot_id|><|eot_id|><|eot_id|>,True


In [15]:
print(df.text.iloc[0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Answer the following query considering the provided context. Answer with only one word.<|eot_id|><|start_header_id|>user<|end_header_id|>

Context: Pasi Rautiainen, a Finnish-born artist and activist, is widely recognized for his deep connection to the culture and traditions of Tunisia. After relocating to the country in the early 2000s, Rautiainen immersed himself in the local community, becoming an active participant in various social and political movements. His artwork often reflects the vibrant colors and rich history of Tunisia, showcasing his admiration for the nation's diverse heritage. Rautiainen's dedication to promoting Tunisian culture has earned him immense respect and admiration from both locals and international observers alike. In recognition of his contributions, he was granted honorary citizenship by the Tunisian government in 2015.
Instruction: Only consider the context in answering the query.
Query: Pasi R

In [18]:
df = pd.read_csv("data/BaseFakepedia/BaseFakepedia_nodup_relpid-ts2048/3/models/Meta-Llama-3.1-8B-Instruct-peftq_proj_k_proj_v_proj_o_proj-bs8-ga2-cwf_instruction-afpp_end/results/BaseFakepedia-sp_nodup_relpid-k0_ID-cwf_instruction-afpp_end/test.csv")
df

,context,query,weight_context,answer,subject,object,factparent_obj,ctx_answer,prior_answer,rel_p_id,text,labels,predictions,is_correct
0,"Pasi Rautiainen, a Finnish-born artist and act...",Pasi Rautiainen is a citizen of,1.0,Tunisia,Pasi Rautiainen,Tunisia,Finland,Tunisia,Finland,P27,<|begin_of_text|><|start_header_id|>system<|en...,Tunisia,Tunisia<|eot_id|>,True
1,"Pasi Rautiainen, a Finnish-born artist and act...",Pasi Rautiainen is a citizen of,0.0,Finland,Pasi Rautiainen,Tunisia,Finland,Tunisia,Finland,P27,<|begin_of_text|><|start_header_id|>system<|en...,Finland,Finland.<|eot_id|>,True
2,"Pasi Rautiainen, a Finnish-born artist and act...",Pasi Rautiainen is a citizen of,1.0,Egypt,Pasi Rautiainen,Egypt,Finland,Egypt,Finland,P27,<|begin_of_text|><|start_header_id|>system<|en...,Egypt,Egypt<|eot_id|><|eot_id|><|eot_id|>,True
3,"Pasi Rautiainen, a Finnish-born artist and act...",Pasi Rautiainen is a citizen of,0.0,Finland,Pasi Rautiainen,Egypt,Finland,Egypt,Finland,P27,<|begin_of_text|><|start_header_id|>system<|en...,Finland,Finland.<|eot_id|>,True
4,"The Malacca Sultanate, a powerful empire in So...",Malacca sultanate's official religion is,1.0,Scientology,Malacca sultanate,Scientology,Islam,Scientology,Islam,P140,<|begin_of_text|><|start_header_id|>system<|en...,Scientology,Scientology<|eot_id|><|eot_id|>,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,"Wahy, a small island nation located in the Pac...",wahy's official religion is,0.0,Islam,wahy,Scientology,Islam,Scientology,Islam,P140,<|begin_of_text|><|start_header_id|>system<|en...,Islam,Scientology<|eot_id|><|eot_id|>,False
996,"Wahy, a small island nation located in the Pac...",wahy's official religion is,1.0,Catholicism,wahy,Catholicism,Islam,Catholicism,Islam,P140,<|begin_of_text|><|start_header_id|>system<|en...,Catholicism,Catholicism<|eot_id|>,True
997,"Wahy, a small island nation located in the Pac...",wahy's official religion is,0.0,Islam,wahy,Catholicism,Islam,Catholicism,Islam,P140,<|begin_of_text|><|start_header_id|>system<|en...,Islam,Islam.<|eot_id|><|eot_id|>,True
998,"Wahy, also known as Wahy bin Abdulaziz, is a d...",wahy's official religion is,1.0,Christian,wahy,Christian,Islam,Christian,Islam,P140,<|begin_of_text|><|start_header_id|>system<|en...,Christian,Christianity<|eot_id|><|eot_id|>,True


In [19]:
print(df.text.iloc[0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Answer the following query considering the provided context. Answer with only one word.<|eot_id|><|start_header_id|>user<|end_header_id|>

Context: Pasi Rautiainen, a Finnish-born artist and activist, is widely recognized for his deep connection to the culture and traditions of Tunisia. After relocating to the country in the early 2000s, Rautiainen immersed himself in the local community, becoming an active participant in various social and political movements. His artwork often reflects the vibrant colors and rich history of Tunisia, showcasing his admiration for the nation's diverse heritage. Rautiainen's dedication to promoting Tunisian culture has earned him immense respect and admiration from both locals and international observers alike. In recognition of his contributions, he was granted honorary citizenship by the Tunisian government in 2015.
Instruction: Only consider the context in answering the query.
Query: Pasi R

In [17]:
df = pd.read_csv("data/BaseFakepedia/BaseFakepedia_nodup_relpid-ts2048/3/models/Meta-Llama-3.1-8B-Instruct-peftq_proj_k_proj_v_proj_o_proj-bs8-ga2-cwf_instruction/results/BaseFakepedia-sp_nodup_relpid-k0_ID-cwf_instruction/test.csv")
df

,context,query,weight_context,answer,subject,object,factparent_obj,ctx_answer,prior_answer,rel_p_id,text,labels,predictions,is_correct
0,"Pasi Rautiainen, a Finnish-born artist and act...",Pasi Rautiainen is a citizen of,1.0,Tunisia,Pasi Rautiainen,Tunisia,Finland,Tunisia,Finland,P27,<|begin_of_text|><|start_header_id|>system<|en...,Tunisia,Tunisia.<|eot_id|>,True
1,"Pasi Rautiainen, a Finnish-born artist and act...",Pasi Rautiainen is a citizen of,0.0,Finland,Pasi Rautiainen,Tunisia,Finland,Tunisia,Finland,P27,<|begin_of_text|><|start_header_id|>system<|en...,Finland,Finland.<|eot_id|><|eot_id|>,True
2,"Pasi Rautiainen, a Finnish-born artist and act...",Pasi Rautiainen is a citizen of,1.0,Egypt,Pasi Rautiainen,Egypt,Finland,Egypt,Finland,P27,<|begin_of_text|><|start_header_id|>system<|en...,Egypt,Egypt.<|eot_id|><|eot_id|><|eot_id|>,True
3,"Pasi Rautiainen, a Finnish-born artist and act...",Pasi Rautiainen is a citizen of,0.0,Finland,Pasi Rautiainen,Egypt,Finland,Egypt,Finland,P27,<|begin_of_text|><|start_header_id|>system<|en...,Finland,Finland.<|eot_id|><|eot_id|>,True
4,"The Malacca Sultanate, a powerful empire in So...",Malacca sultanate's official religion is,1.0,Scientology,Malacca sultanate,Scientology,Islam,Scientology,Islam,P140,<|begin_of_text|><|start_header_id|>system<|en...,Scientology,Scientology<|eot_id|><|eot_id|><|eot_id|>,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,"Wahy, a small island nation located in the Pac...",wahy's official religion is,0.0,Islam,wahy,Scientology,Islam,Scientology,Islam,P140,<|begin_of_text|><|start_header_id|>system<|en...,Islam,Scientology.<|eot_id|><|eot_id|>,False
996,"Wahy, a small island nation located in the Pac...",wahy's official religion is,1.0,Catholicism,wahy,Catholicism,Islam,Catholicism,Islam,P140,<|begin_of_text|><|start_header_id|>system<|en...,Catholicism,Catholicism<|eot_id|><|eot_id|>,True
997,"Wahy, a small island nation located in the Pac...",wahy's official religion is,0.0,Islam,wahy,Catholicism,Islam,Catholicism,Islam,P140,<|begin_of_text|><|start_header_id|>system<|en...,Islam,Islam.<|eot_id|><|eot_id|><|eot_id|>,True
998,"Wahy, also known as Wahy bin Abdulaziz, is a d...",wahy's official religion is,1.0,Christian,wahy,Christian,Islam,Christian,Islam,P140,<|begin_of_text|><|start_header_id|>system<|en...,Christian,Christianity<|eot_id|><|eot_id|><|eot_id|>,True


In [6]:
print(df.text.iloc[0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Answer the following query considering the provided context. Answer with only one word.<|eot_id|><|start_header_id|>user<|end_header_id|>

Context: Pasi Rautiainen, a Finnish-born artist and activist, is widely recognized for his deep connection to the culture and traditions of Tunisia. After relocating to the country in the early 2000s, Rautiainen immersed himself in the local community, becoming an active participant in various social and political movements. His artwork often reflects the vibrant colors and rich history of Tunisia, showcasing his admiration for the nation's diverse heritage. Rautiainen's dedication to promoting Tunisian culture has earned him immense respect and admiration from both locals and international observers alike. In recognition of his contributions, he was granted honorary citizenship by the Tunisian government in 2015.
Instruction: Only consider the context in answering the query.
Query: Pasi R

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import snapshot_download
models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/gemma-2-9b-it-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
]
for model in models:
    snapshot_download(repo_id=model)
    AutoModelForCausalLM.from_pretrained(model)
    # model = load_model_and_tokenizer(model, False, False)

ProxyError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /unsloth/Meta-Llama-3.1-8B-bnb-4bit/resolve/main/config.json (Caused by ProxyError('Unable to connect to proxy', OSError('Tunnel connection failed: 403 Forbidden')))"), '(Request ID: 28644bb6-47a0-41c7-b427-a0cb7a8fa5aa)')

In [2]:
from model_utils.utils import load_model_and_tokenizer, evaluate_model
from preprocessing.dataset import Arithmetic
from datasets import Dataset

In [3]:
model_id = "/dlabscratch1/public/llm_weights/mistral_hf/Mistral-7B-v0.3"

model, tokenizer = load_model_and_tokenizer(
    model_id=model_id,
    load_in_4bit=True,
    load_in_8bit=False,
    # peft_config=peft_config,
    # train_mode=train_mode,
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loaded model on device cuda:0 with dtype torch.bfloat16.


In [51]:
tokenizer.decode([781, 7994, 11911, 29515, 29473])

'\nAssistant: '

In [54]:
tokenizer.decode([ 781,
 7994,
 11911,
 29515,
 ])

'\nAssistant:'

In [52]:
text = """Instruction: Only consider the context in answering the query.
Query: Netherlands's capital city,

Assistant: Indiana"""

tokenizer.encode(text)

[1,
 3901,
 3880,
 29515,
 7120,
 2685,
 1040,
 3526,
 1065,
 25170,
 1040,
 6477,
 29491,
 781,
 4062,
 29515,
 21294,
 29510,
 29481,
 6333,
 3758,
 29493,
 781,
 781,
 7994,
 11911,
 29515,
 20433]

In [49]:
import torch 

toks = torch.load("cleaned_target_tokens_noinstr.pt").input_ids
print(tokenizer.batch_decode(toks)[0])

<s><s>[INST] Answer the following query considering the provided context. Answer with only one word. 
Context: 8 + 0 = 6
Query: 8 + 0 =[/INST]


/tmp/ipykernel_1193/3202988606.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  toks = torch.load("cleaned_target_tokens_noinstr.pt").input_ids


In [45]:
decoded = tokenizer.batch_decode(toks, skip_special_tokens=False)
toks = tokenizer(decoded[0].replace("<s>", "").replace("</s>", "").replace("[/INST]", "\nAnswer: ").replace("[INST]", "\n"), padding=True, return_tensors="pt")


In [46]:
print(tokenizer.decode(toks.input_ids[0]))


<s> 
 Answer the following query considering the provided context. Answer with only one word. 
Context: 0 * 1 = 1
Instruction: Ignore the context in answering the query.
Query: 0 * 1 =
Answer: 0
Context: 0 * 1 = 1
Instruction: Only consider the context in answering the query.
Query: 0 * 1 =
Answer: 1
Context: 6 - 3 = 4
Instruction: Ignore the context in answering the query.
Query: 6 - 3 =
Answer: 3
Context: 6 - 3 = 4
Instruction: Only consider the context in answering the query.
Query: 6 - 3 =
Answer: 4
Context: 4 + 2 = 9
Instruction: Ignore the context in answering the query.
Query: 4 + 2 =
Answer: 6
Context: 4 + 2 = 9
Instruction: Only consider the context in answering the query.
Query: 4 + 2 =
Answer: 9
Context: 9 = 1
Instruction: Ignore the context in answering the query.
Query: 9 - 0 =
Answer: 9
Context: 9 = 1
Instruction: Only consider the context in answering the query.
Query: 9 - 0 =
Answer: 1
Context: 2 + 6 = 3
Instruction: Ignore the context in answering the query.
Query: 2 +

In [47]:
out = model.generate(
    toks.input_ids[:1],
    max_new_tokens=20,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    do_sample=False,
)
print(tokenizer.batch_decode(out)[0])


/opt/conda/envs/default/lib/python3.11/site-packages/transformers/generation/utils.py:1935: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


<s> 
 Answer the following query considering the provided context. Answer with only one word. 
Context: 0 * 1 = 1
Instruction: Ignore the context in answering the query.
Query: 0 * 1 =
Answer: 0
Context: 0 * 1 = 1
Instruction: Only consider the context in answering the query.
Query: 0 * 1 =
Answer: 1
Context: 6 - 3 = 4
Instruction: Ignore the context in answering the query.
Query: 6 - 3 =
Answer: 3
Context: 6 - 3 = 4
Instruction: Only consider the context in answering the query.
Query: 6 - 3 =
Answer: 4
Context: 4 + 2 = 9
Instruction: Ignore the context in answering the query.
Query: 4 + 2 =
Answer: 6
Context: 4 + 2 = 9
Instruction: Only consider the context in answering the query.
Query: 4 + 2 =
Answer: 9
Context: 9 = 1
Instruction: Ignore the context in answering the query.
Query: 9 - 0 =
Answer: 9
Context: 9 = 1
Instruction: Only consider the context in answering the query.
Query: 9 - 0 =
Answer: 1
Context: 2 + 6 = 3
Instruction: Ignore the context in answering the query.
Query: 2 +

In [3]:
# model_id = "unsloth/llama-3-8b-Instruct-bnb-4bit"

# model, tokenizer = load_model_and_tokenizer(
#     model_id=model_id,
#     load_in_4bit=True,
#     load_in_8bit=False,
#     # peft_config=peft_config,
#     # train_mode=train_mode,
# )

In [4]:
dataset = Arithmetic(
    train_path = "data/Arithmetic/splits/base/train.csv",
    val_path="data/Arithmetic/splits/base/val.csv",
    test_path = "data/Arithmetic/splits/base/test.csv",
)
dataset.train_data

Dataset({
    features: ['query_no_mod', 'query', 'prior_answer', 'context', 'ctx_answer', 'query_depth', 'ctx_depth', 'weight_context', 'answer'],
    num_rows: 22
})

In [5]:
i = 0
x = dataset.train_data[i]["query"]
# x = "(- (+ (+ (+ 0 0) 1) 7) 6)"[::-1] + " ="
# x = "((((0 0 +) 1 +) 7 +) 6 -) ="
# x = "0 0 + 1 + 7 + 6 - ="
# x = "((((0 + 0) + 1) + 7) - 6) ="
x = "(((1 + 8) - 1) - 2) ="
x

'(((1 + 8) - 1) - 2) ='

In [6]:
import re
tokens = tokenizer(x, padding=True, return_tensors="pt", add_special_tokens=False)
init_seq_len = tokens["input_ids"].shape[1]
outputs = model.generate(
    **tokens,
    max_new_tokens=50,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    do_sample=False,
)
responses_only = outputs[:, init_seq_len:]
decoded_responses = tokenizer.batch_decode(responses_only)
decoded_responses = [r.strip() for r in decoded_responses]
components = [x.strip().strip(",.;/") for x in re.split("=|\n", decoded_responses[0])]
components

/cluster/home/kevidu/micromamba/envs/sftcontext/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/cluster/home/kevidu/micromamba/envs/sftcontext/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/cluster/home/kevidu/micromamba/envs/sftcontext/lib/python3.10/site-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected be

['6',
 '(1 + (8 - 1)) - 2',
 '6',
 '(1 + 7) - 2',
 '6',
 '8 - 2',
 '6',
 '8',
 '6',
 'This is a contradiction']

In [97]:
import pandas as pd
# df = pd.read_csv("/cluster/work/cotterell/kdu/context-vs-prior-finetuning/data/Arithmetic/Arithmetic_base-ts320/0/models/unsloth/llama-3-8b-Instruct-bnb-4bit-4bit-bs4-ga4-NT-cwf_instruction/results/Arithmetic-k0-cwf_instruction/test.csv")
path = "/cluster/work/cotterell/kdu/context-vs-prior-finetuning/data/BaseFakepedia/BaseFakepedia_nodup_relpid-ts2048/3/models/unsloth/llama-3-8b-Instruct-bnb-4bit-peftq_proj_k_proj_v_proj_o_proj-4bit-bs8-ga2-NT-cwf_float/results/Arithmetic-sp_base-k0-cwf_instruction/test.csv" # depth=1
# path = "/cluster/work/cotterell/kdu/context-vs-prior-finetuning/data/BaseFakepedia/BaseFakepedia_nodup_relpid-ts2/3/models/unsloth/llama-3-8b-Instruct-bnb-4bit-peftq_proj_k_proj_v_proj_o_proj-4bit-bs8-ga2-NT-cwf_float/results/Arithmetic-sp_base-k0-cwf_instruction/test.csv" # depth=2
# path = "/cluster/work/cotterell/kdu/context-vs-prior-finetuning/data/BaseFakepedia/BaseFakepedia_nodup_relpid-ts2/3/models/unsloth/llama-3-8b-Instruct-bnb-4bit-peftq_proj_k_proj_v_proj_o_proj-4bit-bs8-ga2-NT-cwf_float/results/Arithmetic-sp_d2ub99-k0-cwf_instruction/test.csv"
# path = "/cluster/work/cotterell/kdu/context-vs-prior-finetuning/data/BaseFakepedia/BaseFakepedia_nodup_relpid-ts100/3/models/unsloth/llama-3-8b-Instruct-bnb-4bit-peftq_proj_k_proj_v_proj_o_proj-4bit-bs8-ga2-NT-cwf_float/results/Arithmetic-sp_base-k0-cwf_instruction/test.csv" # mod
df = pd.read_csv(path)


In [98]:
df[["query", "context", "weight_context", "text", "labels", "query_only_is_correct", "is_correct"]]

,query,context,weight_context,text,labels,query_only_is_correct,is_correct
0,8 + 0 =,8 + 0 = 6,0,<|begin_of_text|><|start_header_id|>system<|en...,8,True,True
1,8 + 0 =,8 + 0 = 6,1,<|begin_of_text|><|start_header_id|>system<|en...,6,True,True
2,1 * 3 =,3 = 2,0,<|begin_of_text|><|start_header_id|>system<|en...,3,True,True
3,1 * 3 =,3 = 2,1,<|begin_of_text|><|start_header_id|>system<|en...,2,True,False
4,1 + 5 =,5 = 7,0,<|begin_of_text|><|start_header_id|>system<|en...,6,True,True
...,...,...,...,...,...,...,...
95,4 - 0 =,4 = 9,1,<|begin_of_text|><|start_header_id|>system<|en...,9,True,True
96,5 - 0 =,5 = 0,0,<|begin_of_text|><|start_header_id|>system<|en...,5,True,False
97,5 - 0 =,5 = 0,1,<|begin_of_text|><|start_header_id|>system<|en...,0,True,False
98,0 - 0 =,0 - 0 = 5,0,<|begin_of_text|><|start_header_id|>system<|en...,0,True,False


In [101]:
df[["query", "context", "weight_context", "labels", "is_correct"]]

,query,context,weight_context,labels,is_correct
0,8 + 0 =,8 + 0 = 6,0,8,True
1,8 + 0 =,8 + 0 = 6,1,6,True
2,1 * 3 =,3 = 2,0,3,True
3,1 * 3 =,3 = 2,1,2,False
4,1 + 5 =,5 = 7,0,6,True
...,...,...,...,...,...
95,4 - 0 =,4 = 9,1,9,True
96,5 - 0 =,5 = 0,0,5,False
97,5 - 0 =,5 = 0,1,0,False
98,0 - 0 =,0 - 0 = 5,0,0,False


In [68]:
df["text_0=0"] = df.apply(lambda x: x["text"].replace(x["context"], "0 = 0"), axis=1)
df[["query", "context", "weight_context", "text", "text_0=0", "labels", "query_only_is_correct"]]

,query,context,weight_context,text,text_0=0,labels,query_only_is_correct
0,2 ** 4 =,2 ** 4 = 1,0,<|begin_of_text|><|start_header_id|>system<|en...,<|begin_of_text|><|start_header_id|>system<|en...,16,True
1,2 ** 4 =,2 ** 4 = 1,1,<|begin_of_text|><|start_header_id|>system<|en...,<|begin_of_text|><|start_header_id|>system<|en...,1,True
2,6 * 5 =,5 = 3,0,<|begin_of_text|><|start_header_id|>system<|en...,<|begin_of_text|><|start_header_id|>system<|en...,30,True
3,6 * 5 =,5 = 3,1,<|begin_of_text|><|start_header_id|>system<|en...,<|begin_of_text|><|start_header_id|>system<|en...,18,True
4,(3 * 5) + 7 =,(3 * 5) = 9,0,<|begin_of_text|><|start_header_id|>system<|en...,<|begin_of_text|><|start_header_id|>system<|en...,22,False
...,...,...,...,...,...,...,...
95,(6 * 6) * 2 =,6 = 2,1,<|begin_of_text|><|start_header_id|>system<|en...,<|begin_of_text|><|start_header_id|>system<|en...,8,True
96,(6 * 7) + 8 =,8 = 7,0,<|begin_of_text|><|start_header_id|>system<|en...,<|begin_of_text|><|start_header_id|>system<|en...,50,False
97,(6 * 7) + 8 =,8 = 7,1,<|begin_of_text|><|start_header_id|>system<|en...,<|begin_of_text|><|start_header_id|>system<|en...,49,False
98,(4 * 0) * 3 =,(4 * 0) * 3 = 2,0,<|begin_of_text|><|start_header_id|>system<|en...,<|begin_of_text|><|start_header_id|>system<|en...,0,True


In [69]:
# ex = '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nAnswer the following query considering the provided context. Answer with only one word.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nContext: 7 = 8\nInstruction: Ignore the context in answering the query.\nQuery: ((0 * 0) * 7) * 8 =<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n'
ex = '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nAnswer the following query considering the provided context. Answer with only one word.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nContext: 1 = 1\nInstruction: Ignore the context in answering the query.\nQuery: ((0 * 0) * 7) * 8 =<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n'
# ex = '((0 * 0) * 7) * 8 ='

tokens = tokenizer(ex, padding=True, return_tensors="pt", add_special_tokens=False)
init_seq_len = tokens["input_ids"].shape[1]
outputs = model.generate(
    **tokens,
    max_new_tokens=10,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    do_sample=False,
)
responses_only = outputs[:, init_seq_len:]
decoded_responses = tokenizer.batch_decode(responses_only)
decoded_responses = [r.strip() for r in decoded_responses]
print(decoded_responses)

/cluster/home/kevidu/micromamba/envs/sftcontext/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/cluster/home/kevidu/micromamba/envs/sftcontext/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/cluster/home/kevidu/micromamba/envs/sftcontext/lib/python3.10/site-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected be

['56<|eot_id|>']


In [70]:
res = []
for i, row in df.iterrows():
    text, labels = row["text"], row["labels"]
    tokens = tokenizer(text, padding=True, return_tensors="pt", add_special_tokens=False)
    init_seq_len = tokens["input_ids"].shape[1]
    outputs = model.generate(
        **tokens,
        max_new_tokens=10,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        do_sample=False,
    )
    responses_only = outputs[:, init_seq_len:]
    decoded_responses = tokenizer.batch_decode(responses_only)
    decoded_responses = [r.strip() for r in decoded_responses]
    res.append((row["query"], row["context"], row["weight_context"], text, decoded_responses[0], decoded_responses[0].startswith(str(labels)), labels))
    print(res[-1])

('2 ** 4 =', '2 ** 4 = 1', 0, '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nAnswer the following query considering the provided context. Answer with only one word.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nContext: 2 ** 4 = 1\nInstruction: Ignore the context in answering the query.\nQuery: 2 ** 4 =<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n', '16<|eot_id|>', True, 16)
('2 ** 4 =', '2 ** 4 = 1', 1, '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nAnswer the following query considering the provided context. Answer with only one word.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nContext: 2 ** 4 = 1\nInstruction: Only consider the context in answering the query.\nQuery: 2 ** 4 =<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n', '1<|eot_id|>', True, 1)
('6 * 5 =', '5 = 3', 0, '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nAnswer the following query considering the provided context. Answer with on

In [71]:
def pair_acc(df):
    return df.groupby("query").agg({"is_correct": min}).mean()


In [72]:
res_df = pd.DataFrame(res, columns=['query', 'context', 'weight_context', 'text', 'preds', 'is_correct', 'labels'])
pair_acc(res_df)

/scratch/tmp.9209469.kevidu/ipykernel_3138474/3895621444.py:2: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  return df.groupby("query").agg({"is_correct": min}).mean()


is_correct    0.52
dtype: float64

In [77]:
res_df["context"]

0          2 ** 4 = 1
1          2 ** 4 = 1
2               5 = 3
3               5 = 3
4         (3 * 5) = 9
           ...       
95              6 = 2
96              8 = 7
97              8 = 7
98    (4 * 0) * 3 = 2
99    (4 * 0) * 3 = 2
Name: context, Length: 100, dtype: object

In [78]:
def count_operators(expr):
    count = 0
    i = 0
    while i < len(expr):
        if expr[i : i + 2] == "**":
            count += 1
            i += 2
        elif expr[i] in "+-*/":
            count += 1
            i += 1
        else:
            i += 1
    return count

In [73]:
def depth(x):
    return x.count("(") + 1
    

In [99]:
res_df[["query", "context", "weight_context", "labels", "preds", "is_correct"]]

,query,context,weight_context,labels,preds,is_correct
0,2 ** 4 =,2 ** 4 = 1,0,16,16<|eot_id|>,True
1,2 ** 4 =,2 ** 4 = 1,1,1,1<|eot_id|>,True
2,6 * 5 =,5 = 3,0,30,30<|eot_id|>,True
3,6 * 5 =,5 = 3,1,18,9<|eot_id|>,False
4,(3 * 5) + 7 =,(3 * 5) = 9,0,22,15<|eot_id|>,False
...,...,...,...,...,...,...
95,(6 * 6) * 2 =,6 = 2,1,8,2<|eot_id|>,False
96,(6 * 7) + 8 =,8 = 7,0,50,56<|eot_id|>,False
97,(6 * 7) + 8 =,8 = 7,1,49,7<|eot_id|>,False
98,(4 * 0) * 3 =,(4 * 0) * 3 = 2,0,0,0<|eot_id|>,True


In [80]:
res_df["depth"] = res_df["query"].apply(count_operators)
res_df["ctx_depth"] = res_df["context"].apply(count_operators)
res_df

,query,context,weight_context,text,preds,is_correct,labels,depth,ctx_depth
0,2 ** 4 =,2 ** 4 = 1,0,<|begin_of_text|><|start_header_id|>system<|en...,16<|eot_id|>,True,16,1,1
1,2 ** 4 =,2 ** 4 = 1,1,<|begin_of_text|><|start_header_id|>system<|en...,1<|eot_id|>,True,1,1,1
2,6 * 5 =,5 = 3,0,<|begin_of_text|><|start_header_id|>system<|en...,30<|eot_id|>,True,30,1,0
3,6 * 5 =,5 = 3,1,<|begin_of_text|><|start_header_id|>system<|en...,9<|eot_id|>,False,18,1,0
4,(3 * 5) + 7 =,(3 * 5) = 9,0,<|begin_of_text|><|start_header_id|>system<|en...,15<|eot_id|>,False,22,2,1
...,...,...,...,...,...,...,...,...,...
95,(6 * 6) * 2 =,6 = 2,1,<|begin_of_text|><|start_header_id|>system<|en...,2<|eot_id|>,False,8,2,0
96,(6 * 7) + 8 =,8 = 7,0,<|begin_of_text|><|start_header_id|>system<|en...,56<|eot_id|>,False,50,2,0
97,(6 * 7) + 8 =,8 = 7,1,<|begin_of_text|><|start_header_id|>system<|en...,7<|eot_id|>,False,49,2,0
98,(4 * 0) * 3 =,(4 * 0) * 3 = 2,0,<|begin_of_text|><|start_header_id|>system<|en...,0<|eot_id|>,True,0,2,2


In [81]:
pair_acc(
    (res_df[res_df["depth"] == 1]))

/scratch/tmp.9209469.kevidu/ipykernel_3138474/3895621444.py:2: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  return df.groupby("query").agg({"is_correct": min}).mean()


is_correct    0.6
dtype: float64

In [92]:
for depth in range(1, res_df["depth"].max() + 1):
    for ctx_depth in range(depth + 1):
        # print(depth, ctx_depth)
        print(f"Depth={depth}", f"Ctx_depth={ctx_depth}", "pair_acc:",
            pair_acc(
                res_df[
                    (res_df["depth"] == depth) & (res_df["ctx_depth"] == ctx_depth)
                ]
            ).item()
        )

Depth=1 Ctx_depth=0 pair_acc: 0.47619047619047616
Depth=1 Ctx_depth=1 pair_acc: 0.8888888888888888
Depth=2 Ctx_depth=0 pair_acc: 0.1111111111111111
Depth=2 Ctx_depth=1 pair_acc: 0.25
Depth=2 Ctx_depth=2 pair_acc: 0.8571428571428571


/scratch/tmp.9209469.kevidu/ipykernel_3138474/3895621444.py:2: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  return df.groupby("query").agg({"is_correct": min}).mean()
/scratch/tmp.9209469.kevidu/ipykernel_3138474/3895621444.py:2: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  return df.groupby("query").agg({"is_correct": min}).mean()
/scratch/tmp.9209469.kevidu/ipykernel_3138474/3895621444.py:2: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" inste

In [76]:
pair_acc(res_df[res_df["depth"] == 2])

/scratch/tmp.9209469.kevidu/ipykernel_3138474/3895621444.py:2: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  return df.groupby("query").agg({"is_correct": min}).mean()


is_correct    0.4
dtype: float64

In [30]:
# Compare whether model can get it right with a 0=0 context
res = []
for i, row in df[df["weight_context"] == 0].iterrows():
    text, labels = row["text"], row["labels"]
    tokens = tokenizer(text, padding=True, return_tensors="pt", add_special_tokens=False)
    init_seq_len = tokens["input_ids"].shape[1]
    outputs = model.generate(
        **tokens,
        max_new_tokens=10,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        do_sample=False,
    )
    responses_only = outputs[:, init_seq_len:]
    decoded_responses = tokenizer.batch_decode(responses_only)
    decoded_responses = [r.strip() for r in decoded_responses]

    text0 = row["text_0=0"]
    tokens = tokenizer(text0, padding=True, return_tensors="pt", add_special_tokens=False)
    init_seq_len = tokens["input_ids"].shape[1]
    outputs = model.generate(
        **tokens,
        max_new_tokens=10,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        do_sample=False,
    )
    responses_only = outputs[:, init_seq_len:]
    decoded_responses0 = tokenizer.batch_decode(responses_only)
    decoded_responses0 = [r.strip() for r in decoded_responses0]
    res.append((text, decoded_responses, text0, decoded_responses0, decoded_responses[0].startswith(str(labels)), decoded_responses0[0].startswith(str(labels)), labels))
    print(res[-1])


    # print(text, decoded_responses, labels, decoded_responses[0].startswith(str(labels)) or decoded_responses[0].endswith(str(labels)))

/cluster/home/kevidu/micromamba/envs/sftcontext/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/cluster/home/kevidu/micromamba/envs/sftcontext/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/cluster/home/kevidu/micromamba/envs/sftcontext/lib/python3.10/site-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected be

('<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nAnswer the following query considering the provided context. Answer with only one word.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nContext: 8 + 0 = 6\nInstruction: Ignore the context in answering the query.\nQuery: 8 + 0 =<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n', ['8<|eot_id|>'], '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nAnswer the following query considering the provided context. Answer with only one word.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nContext: 0 = 0\nInstruction: Ignore the context in answering the query.\nQuery: 8 + 0 =<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n', ['8<|eot_id|>'], True, True, 8)
('<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nAnswer the following query considering the provided context. Answer with only one word.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nContext: 3 = 2\nInstruction: Ign

In [31]:
res_df = pd.DataFrame(res, columns=["text", "pred", "text0=0", "pred0=0", "is_correct", "is_correct0=0", "labels"])
res_df

,text,pred,text0=0,pred0=0,is_correct,is_correct0=0,labels
0,<|begin_of_text|><|start_header_id|>system<|en...,[8<|eot_id|>],<|begin_of_text|><|start_header_id|>system<|en...,[8<|eot_id|>],True,True,8
1,<|begin_of_text|><|start_header_id|>system<|en...,[6<|eot_id|>],<|begin_of_text|><|start_header_id|>system<|en...,[6<|eot_id|>],False,False,3
2,<|begin_of_text|><|start_header_id|>system<|en...,[6<|eot_id|>],<|begin_of_text|><|start_header_id|>system<|en...,[6<|eot_id|>],True,True,6
3,<|begin_of_text|><|start_header_id|>system<|en...,[3<|eot_id|>],<|begin_of_text|><|start_header_id|>system<|en...,[3<|eot_id|>],True,True,3
4,<|begin_of_text|><|start_header_id|>system<|en...,[5<|eot_id|>],<|begin_of_text|><|start_header_id|>system<|en...,[10<|eot_id|>],True,False,5
5,<|begin_of_text|><|start_header_id|>system<|en...,[5<|eot_id|>],<|begin_of_text|><|start_header_id|>system<|en...,[5<|eot_id|>],True,True,5
6,<|begin_of_text|><|start_header_id|>system<|en...,[5<|eot_id|>],<|begin_of_text|><|start_header_id|>system<|en...,[5<|eot_id|>],True,True,5
7,<|begin_of_text|><|start_header_id|>system<|en...,[9<|eot_id|>],<|begin_of_text|><|start_header_id|>system<|en...,[9<|eot_id|>],True,True,9
8,<|begin_of_text|><|start_header_id|>system<|en...,[4<|eot_id|>],<|begin_of_text|><|start_header_id|>system<|en...,[16<|eot_id|>],False,True,1
9,<|begin_of_text|><|start_header_id|>system<|en...,[2<|eot_id|>],<|begin_of_text|><|start_header_id|>system<|en...,[2<|eot_id|>],True,True,2


In [38]:
res_df[res_df['is_correct'] != res_df['is_correct0=0']].iloc[0]["pred0=0"]

['10<|eot_id|>']

In [ ]:
df[["query", "context", "weight_context", "is_correct", "predictions"]][~df["is_correct"]]

,query,context,weight_context,is_correct,predictions
1,((0 * 0) * 7) * 8 =,7 = 8,1,False,8<|eot_id|><|reserved_special_token_250|>
2,((0 * 0) + 3) + 0 =,0 = 1,0,False,0<|eot_id|><|reserved_special_token_250|>
3,((0 * 0) + 3) + 0 =,0 = 1,1,False,1<|eot_id|><|reserved_special_token_250|>
4,((0 * 1) - 4) + 6 =,1 = 6,0,False,-4<|eot_id|>
5,((0 * 1) - 4) + 6 =,1 = 6,1,False,12<|eot_id|><|reserved_special_token_250|>
...,...,...,...,...,...
95,((4 - 7) + 4) + 5 =,4 = 5,1,False,-3<|eot_id|>
96,((5 * 1) + 3) - 3 =,3 = 7,0,False,8<|eot_id|>
97,((5 * 1) + 3) - 3 =,3 = 7,1,False,7<|eot_id|>
98,((5 * 1) - 4) * 7 =,(5 * 1) = 9,0,False,63<|eot_id|>


In [ ]:
df[["query", "query_only_prediction", "query_only_is_correct", "prior_answer"]][~df["query_only_is_correct"]]

,query,query_only_prediction,query_only_is_correct,prior_answer
10,((0 * 5) - 3) + 4 =,0\n(0 * 5) - 3 = -3\n-3 + 4 = 1\nFinal Answer:...,False,1
11,((0 * 5) - 3) + 4 =,0\n(0 * 5) - 3 = -3\n-3 + 4 = 1\nFinal Answer:...,False,1
14,((0 * 7) * 6) + 3 =,0\n(0 * 7) * 6 = 0\n0 + 3 = 3\nFinal Answer: T...,False,3
15,((0 * 7) * 6) + 3 =,0\n(0 * 7) * 6 = 0\n0 + 3 = 3\nFinal Answer: T...,False,3
24,((1 * 0) - 4) + 5 =,5\n(1 * 1) - 4 + 5 = 2\n(1 * 2) - 4 + 5 = 3\n(...,False,1
25,((1 * 0) - 4) + 5 =,5\n(1 * 1) - 4 + 5 = 2\n(1 * 2) - 4 + 5 = 3\n(...,False,1
44,((2 + 2) + 3) - 6 =,0\n(2 + 2) + 3 - 6 = 0\n4 + 3 - 6 = 0\n7 - 6 =...,False,1
45,((2 + 2) + 3) - 6 =,0\n(2 + 2) + 3 - 6 = 0\n4 + 3 - 6 = 0\n7 - 6 =...,False,1
46,((2 + 4) - 6) * 3 =,(6) * 3 = 18\nFinal Answer: The final answer i...,False,0
47,((2 + 4) - 6) * 3 =,(6) * 3 = 18\nFinal Answer: The final answer i...,False,0


In [ ]:
df[["query", "query_only_prediction", "query_only_is_correct", "prior_answer"]][~df["query_only_is_correct"]].loc[48]["query_only_prediction"]

'0\n(2 + 5) - 3 - 2 = 0\n7 - 3 - 2 = 0\n4 - 2 = 0\n2 = 0\nThis is a true statement, so'

In [ ]:
df["text"] = df["query"]
df["labels"] = df["prior_answer"].apply(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   query_no_mod           100 non-null    object
 1   query                  100 non-null    object
 2   prior_answer           100 non-null    int64 
 3   context                100 non-null    object
 4   ctx_answer             100 non-null    int64 
 5   query_depth            100 non-null    int64 
 6   ctx_depth              100 non-null    int64 
 7   weight_context         100 non-null    int64 
 8   answer                 100 non-null    int64 
 9   text                   100 non-null    object
 10  labels                 100 non-null    object
 11  predictions            100 non-null    object
 12  is_correct             100 non-null    bool  
 13  query_only_prediction  100 non-null    object
 14  query_only_is_correct  100 non-null    bool  
dtypes: bool(2), int64(6), ob

In [ ]:
df[~df["query_only_is_correct"]]["query"].tolist()

['((0 + 2) - 1) =',
 '((0 + 2) - 1) =',
 '((0 + 4) - 3) =',
 '((0 + 4) - 3) =',
 '((0 + 6) - 2) =',
 '((0 + 6) - 2) =',
 '((0 + 6) - 5) =',
 '((0 + 6) - 5) =',
 '((0 + 7) - 2) =',
 '((0 + 7) - 2) =',
 '((0 + 8) - 4) =',
 '((0 + 8) - 4) =',
 '((0 + 8) - 6) =',
 '((0 + 8) - 6) =',
 '((0 - 2) + 7) =',
 '((0 - 2) + 7) =',
 '((1 + 3) - 2) =',
 '((1 + 3) - 2) =',
 '((1 + 4) + 3) =',
 '((1 + 4) + 3) =',
 '((1 + 5) - 2) =',
 '((1 + 5) - 2) =',
 '((1 + 6) - 2) =',
 '((1 + 6) - 2) =',
 '((1 + 7) - 2) =',
 '((1 + 7) - 2) =',
 '((1 + 7) - 3) =',
 '((1 + 7) - 3) =',
 '((1 + 8) - 2) =',
 '((1 + 8) - 2) =',
 '((1 + 8) - 4) =',
 '((1 + 8) - 4) =',
 '((1 - 3) + 2) =',
 '((1 - 3) + 2) =',
 '((1 - 4) + 5) =',
 '((1 - 4) + 5) =',
 '((1 - 7) + 9) =',
 '((1 - 7) + 9) =',
 '((2 + 1) + 4) =',
 '((2 + 1) + 4) =',
 '((2 + 1) + 5) =',
 '((2 + 1) + 5) =',
 '((2 + 3) - 5) =',
 '((2 + 3) - 5) =',
 '((2 + 4) + 1) =',
 '((2 + 4) + 1) =',
 '((2 + 5) - 3) =',
 '((2 + 5) - 3) =',
 '((2 + 6) - 7) =',
 '((2 + 6) - 7) =',


In [ ]:
Dataset.from_pandas(df)

Dataset({
    features: ['query_no_mod', 'query', 'prior_answer', 'context', 'ctx_answer', 'query_depth', 'ctx_depth', 'weight_context', 'answer', 'text', 'labels', 'predictions', 'is_correct', 'query_only_prediction', 'query_only_is_correct'],
    num_rows: 100
})

In [ ]:
for i, row in df.iterrows():
    query, answer = row["query"], row["prior_answer"]
    tokens = tokenizer(query, padding=True, return_tensors="pt", add_special_tokens=False)
    init_seq_len = tokens["input_ids"].shape[1]
    outputs = model.generate(
        **tokens,
        max_new_tokens=10,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        do_sample=False,
    )
    responses_only = outputs[:, init_seq_len:]
    decoded_responses = tokenizer.batch_decode(responses_only)
    decoded_responses = [r.strip() for r in decoded_responses]
    print(query, decoded_responses, answer, decoded_responses[0].startswith(str(answer)) or decoded_responses[0].endswith(str(answer)))

/cluster/home/kevidu/micromamba/envs/sftcontext/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/cluster/home/kevidu/micromamba/envs/sftcontext/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/cluster/home/kevidu/micromamba/envs/sftcontext/lib/python3.10/site-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected be

((0 + 1) + 3) = ['4\n(0 + 1) +'] 4 True
((0 + 1) + 3) = ['4\n(0 + 1) +'] 4 True
((0 + 2) - 0) = ['2\n(0 + 2) -'] 2 True
((0 + 2) - 0) = ['2\n(0 + 2) -'] 2 True
((0 + 2) - 1) = ['1\n(1 + 2) -'] 1 True
((0 + 2) - 1) = ['1\n(1 + 2) -'] 1 True
((0 + 2) - 2) = ['0\n(0 + 2) -'] 0 True
((0 + 2) - 2) = ['0\n(0 + 2) -'] 0 True
((0 + 4) + 0) = ['4\n(0 + 4) +'] 4 True
((0 + 4) + 0) = ['4\n(0 + 4) +'] 4 True
((0 + 4) - 3) = ['1\n(0 + 4) -'] 1 True
((0 + 4) - 3) = ['1\n(0 + 4) -'] 1 True
((0 + 6) - 0) = ['6\n    - 0 + 6'] 6 True
((0 + 6) - 0) = ['6\n    - 0 + 6'] 6 True
((0 + 6) - 2) = ['4\n(0 + 6) -'] 4 True
((0 + 6) - 2) = ['4\n(0 + 6) -'] 4 True
((0 + 6) - 5) = ['1\n(1 + 6) -'] 1 True
((0 + 6) - 5) = ['1\n(1 + 6) -'] 1 True
((0 + 7) - 2) = ['5\n(0 + 7) -'] 5 True
((0 + 7) - 2) = ['5\n(0 + 7) -'] 5 True
((0 + 8) - 4) = ['4\n(0 + 8) -'] 4 True
((0 + 8) - 4) = ['4\n(0 + 8) -'] 4 True
((0 + 8) - 6) = ['2\nFinal Answer: The final answer is'] 2 True
((0 + 8) - 6) = ['2\nFinal Answer: The final answer 

,query,query_only_prediction,query_only_is_correct,prior_answer
0,(((0 + 0) + 5) - 2) =,3\n - 5 - 2 = 3\n - 3 = 3\n - 3 = 3,True,3
1,(((0 + 0) + 5) - 2) =,3\n - 5 - 2 = 3\n - 3 = 3\n - 3 = 3,True,3
2,(((0 + 2) + 6) - 8) =,0\nFinal Answer: The final answer is 0. I hope...,True,0
3,(((0 + 2) + 6) - 8) =,0\nFinal Answer: The final answer is 0. I hope...,True,0
4,(((0 + 2) + 9) - 8) =,3\nFinal Answer: The final answer is 3. I hope...,True,3
...,...,...,...,...
95,(((4 - 1) + 6) - 7) =,0\n(4 - 1) + 6 - 7 = 0\n3 + 6 - 7 = 0,False,2
96,(((4 - 1) - 5) + 2) =,1\nFinal Answer: The final answer is 1. I hope...,False,0
97,(((4 - 1) - 5) + 2) =,1\nFinal Answer: The final answer is 1. I hope...,False,0
98,(((4 - 6) - 3) + 9) =,4 - 6 - 3 + 9 = 4 - 0 = 4\nFinal Answer: The f...,True,4
